In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
import cv2
import matplotlib.pyplot as plt
import h5py
import wandb
import random


# Configure Keras to use GPU
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config=config)
tf.compat.v1.keras.backend.set_session(sess)



2024-04-01 13:06:21.196551: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-01 13:06:21.230554: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-01 13:06:21.230582: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-01 13:06:21.231548: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-01 13:06:21.237114: I tensorflow/core/platform/cpu_feature_guar

In [23]:
import wandb
import tensorflow as tf
import numpy as np
import h5py
from tensorflow.keras import layers, models, optimizers

# Ensure wandb is installed: pip install wandb
# Initialize your W&B project before running the script

# Dataset loading and preparation
def load_and_prepare_dataset(batch_size):
    with h5py.File('TrainingData5zeroes.h5', 'r') as hdf:
        images = np.array(hdf.get('images'))
        boxes = np.array(hdf.get('boxes'))

    image_normalized = (images + 1e-9) / 9.26
    normalized_boxes = boxes / [1, 64, 64, 64, 64]

    images_np = image_normalized
    probabilities = np.array(normalized_boxes[:, :, :-4])
    probabilities = tf.expand_dims(probabilities, axis=1)
    boxes_np = np.array(normalized_boxes[:, :, 1:])
    boxes_np = tf.expand_dims(boxes_np, axis=1)

    dataset = tf.data.Dataset.from_tensor_slices((images_np, {'x_prob_reshape': probabilities, 'x_boxes_reshape': boxes_np}))
    dataset = dataset.shuffle(buffer_size=10000).batch(batch_size)
    return dataset

# Model building
def build_model():
    input_shape = (64, 64, 1)
    num_classes = 280
    num_coordinates = 4

    x_input = layers.Input(shape=input_shape)
    #Layer 1
    x = layers.Conv2D(64, kernel_size=3, padding='same', activation='relu')(x_input)
    x = layers.MaxPool2D()(x)
    x = layers.BatchNormalization()(x) 
    
    x = layers.Conv2D(64, kernel_size=3, padding='same', activation='relu')(x)
    #Layer 2
    x = layers.Conv2D(128, kernel_size=3, padding='same', activation='relu')(x)
    x = layers.Conv2D(128, kernel_size=3, padding='same', activation='relu')(x)
    #Layer 3
    x = layers.Conv2D(256, kernel_size=3, padding='same', activation='relu')(x)
    x = layers.Conv2D(256, kernel_size=3, padding='same', activation='relu')(x)
    #Layer 4
    x = layers.Conv2D(512, kernel_size=3, padding='same', activation='relu')(x)
    x = layers.MaxPool2D()(x)
    x = layers.Conv2D(512, kernel_size=3, padding='same', activation='relu')(x)
    x = layers.MaxPool2D()(x)
    x = layers.Conv2D(512, kernel_size=3, padding='same', activation='relu')(x)
    x = layers.MaxPool2D()(x)
    #Layer 5
    x = layers.Conv2D(256, kernel_size=5, padding='same', activation='relu')(x)
    x = layers.MaxPool2D()(x)
    x = layers.BatchNormalization()(x) 
    
    
    x = layers.Flatten()(x)
    x_prob = layers.Dense(num_classes, activation='sigmoid', name='x_prob')(x)
    x_boxes = layers.Dense(num_classes * num_coordinates, activation='sigmoid', name='x_boxes')(x)
    x_prob_reshape = layers.Reshape((-1, num_classes, 1), name='x_prob_reshape')(x_prob)
    x_boxes_reshape = layers.Reshape((-1, num_classes, num_coordinates), name='x_boxes_reshape')(x_boxes)

    model = models.Model(x_input, [x_prob_reshape, x_boxes_reshape])
    return model

# Optimizer selection
def select_optimizer(optimizer_name, learning_rate):
    if optimizer_name == 'adam':
        return optimizers.Adam(learning_rate=learning_rate)
    elif optimizer_name == 'sgd':
        return optimizers.SGD(learning_rate=learning_rate, momentum=0.9)
    else:
        raise ValueError("Unsupported optimizer specified.")

# Training function with sweep
def train_with_sweep():
    # config_defaults = {
    #     'batch_size': 32,
    #     'epochs': 10,
    #     'optimizer': 'adam',
    #     'learning_rate': 0.001
    # }
    with wandb.init():
        config = wandb.config

        dataset = load_and_prepare_dataset(config.batch_size)
        model = build_model()
        # model = tf.keras.models.load_model('M11overfittedmodel3variant.h5')
        optimizer = select_optimizer(config.optimizer, config.learning_rate)
        model.compile(optimizer=optimizer, 
                      loss={'x_prob_reshape': 'binary_crossentropy', 'x_boxes_reshape': 'mean_squared_error'}, 
                      metrics={'x_prob_reshape': 'accuracy'})

        # Fit the model
        model.fit(dataset, epochs=config.epochs, callbacks=[wandb.keras.WandbCallback()])


In [25]:

sweep_config = {
    'method': 'random'
    }
metric = {
    'name': 'loss',
    'goal': 'minimize'   
    }

sweep_config['metric'] = metric
parameters_dict = {
    'optimizer': {
        'values': ['adam', 'sgd']
        }
    # 'fc_layer_size': {
    #     'values': [128, 256, 512]
    #     },
    # 'dropout': {
    #       'values': [0.3, 0.4, 0.5]
    #     },
    }

sweep_config['parameters'] = parameters_dict
parameters_dict.update({
    'epochs': {
        'value': 10}
    })
parameters_dict.update({
    'learning_rate': {
        # a flat distribution between 0 and 0.1
        'distribution': 'uniform',
        'min': 0,
        'max': 0.1
      },
    'batch_size': {
        'values': [32, 64, 256]
      }
    })

In [21]:
sweep_config


{'method': 'random',
 'metric': {'name': 'loss', 'goal': 'minimize'},
 'parameters': {'optimizer': {'values': ['adam', 'sgd']},
  'epochs': {'value': 11},
  'learning_rate': {'distribution': 'uniform', 'min': 0, 'max': 0.1},
  'batch_size': {'values': [32, 64, 128]}}}

In [26]:
sweep_id = wandb.sweep(sweep_config, project="Working Hyperparameter Tuning")
wandb.agent(sweep_id, train_with_sweep, count=5)

Create sweep with ID: egosie6n
Sweep URL: https://wandb.ai/alphanium/Working%20Hyperparameter%20Tuning/sweeps/egosie6n


wandb: Agent Starting Run: qngp3s0s with config:
wandb: 	batch_size: 32
wandb: 	epochs: 10
wandb: 	learning_rate: 0.028461417967800853
wandb: 	optimizer: sgd
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Epoch 1/10
313/313 [==============================] - 10s 24ms/step - loss: 0.2561 - x_prob_reshape_loss: 0.1769 - x_boxes_reshape_loss: 0.0792 - x_prob_reshape_accuracy: 0.9293
Epoch 2/10
313/313 [==============================] - 8s 24ms/step - loss: 0.1606 - x_prob_reshape_loss: 0.1084 - x_boxes_reshape_loss: 0.0522 - x_prob_reshape_accuracy: 0.9553
Epoch 3/10
313/313 [==============================] - 8s 24ms/step - loss: 0.1434 - x_prob_reshape_loss: 0.0938 - x_boxes_reshape_loss: 0.0496 - x_prob_reshape_accuracy: 0.9606
Epoch 4/10
313/313 [==============================] - 8s 25ms/step - loss: 0.1336 - x_prob_reshape_loss: 0.0856 - x_boxes_reshape_loss: 0.0481 - x_prob_reshape_accuracy: 0.9636
Epoch 5/10
313/313 [==============================] - 8s 25ms/step - loss: 0.1318 - x_prob_reshape_loss: 0.0841 - x_boxes_reshape_loss: 0.0477 - x_prob_reshape_accuracy: 0.9642
Epoch 6/10
313/313 [==============================] - 8s 24ms/step - loss: 0.1198 - x_prob_reshape_loss: 0.0733 - 

epoch,▁▂▃▃▄▅▆▆▇█
loss,█▃▃▂▂▂▁▂▁▁
x_boxes_reshape_loss,█▂▂▂▂▁▁▁▁▁
x_prob_reshape_accuracy,▁▅▆▇▇▇▇▇██
x_prob_reshape_loss,█▄▃▂▂▂▂▂▁▁
epoch,9
loss,0.10873
x_boxes_reshape_loss,0.04489
x_prob_reshape_accuracy,0.97293
x_prob_reshape_loss,0.06384


wandb: Agent Starting Run: sb7lvyqc with config:
wandb: 	batch_size: 64
wandb: 	epochs: 10
wandb: 	learning_rate: 0.029841371945855368
wandb: 	optimizer: adam
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


...

Epoch 1/10
40/40 [==============================] - 23s 131ms/step - loss: 1.0701 - x_prob_reshape_loss: 0.9319 - x_boxes_reshape_loss: 0.1383 - x_prob_reshape_accuracy: 0.7433
Epoch 2/10
40/40 [==============================] - 5s 131ms/step - loss: 0.5761 - x_prob_reshape_loss: 0.4833 - x_boxes_reshape_loss: 0.0928 - x_prob_reshape_accuracy: 0.7666
Epoch 3/10
40/40 [==============================] - 5s 132ms/step - loss: 0.5195 - x_prob_reshape_loss: 0.4428 - x_boxes_reshape_loss: 0.0767 - x_prob_reshape_accuracy: 0.7798
Epoch 4/10
40/40 [==============================] - 5s 132ms/step - loss: 0.5140 - x_prob_reshape_loss: 0.4398 - x_boxes_reshape_loss: 0.0742 - x_prob_reshape_accuracy: 0.7812
Epoch 5/10
40/40 [==============================] - 5s 131ms/step - loss: 0.5139 - x_prob_reshape_loss: 0.4401 - x_boxes_reshape_loss: 0.0738 - x_prob_reshape_accuracy: 0.7812
Epoch 6/10
40/40 [==============================] - 5s 129ms/step - loss: 0.5113 - x_prob_reshape_loss: 0.4389 - x_boxe

epoch,▁▂▃▃▄▅▆▆▇█
loss,█▂▁▁▁▁▁▁▁▁
x_boxes_reshape_loss,█▃▂▁▁▁▁▁▁▁
x_prob_reshape_accuracy,▁▅████████
x_prob_reshape_loss,█▂▁▁▁▁▁▁▁▁
epoch,9
loss,0.50859
x_boxes_reshape_loss,0.07097
x_prob_reshape_accuracy,0.78253
x_prob_reshape_loss,0.43762


wandb: Agent Starting Run: qh2267ds with config:
wandb: 	batch_size: 64
wandb: 	epochs: 10
wandb: 	learning_rate: 0.09087766902318978
wandb: 	optimizer: sgd
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Epoch 1/10
157/157 [==============================] - 8s 41ms/step - loss: 0.2365 - x_prob_reshape_loss: 0.1607 - x_boxes_reshape_loss: 0.0759 - x_prob_reshape_accuracy: 0.9359
Epoch 2/10
157/157 [==============================] - 6s 41ms/step - loss: 0.1356 - x_prob_reshape_loss: 0.0850 - x_boxes_reshape_loss: 0.0506 - x_prob_reshape_accuracy: 0.9663
Epoch 3/10
157/157 [==============================] - 6s 41ms/step - loss: 0.1254 - x_prob_reshape_loss: 0.0766 - x_boxes_reshape_loss: 0.0488 - x_prob_reshape_accuracy: 0.9682
Epoch 4/10
157/157 [==============================] - 6s 41ms/step - loss: 0.1221 - x_prob_reshape_loss: 0.0743 - x_boxes_reshape_loss: 0.0478 - x_prob_reshape_accuracy: 0.9687
Epoch 5/10
157/157 [==============================] - 6s 41ms/step - loss: 0.1132 - x_prob_reshape_loss: 0.0664 - x_boxes_reshape_loss: 0.0468 - x_prob_reshape_accuracy: 0.9719
Epoch 6/10
157/157 [==============================] - 6s 41ms/step - loss: 0.1160 - x_prob_reshape_loss: 0.0696 - x

epoch,▁▂▃▃▄▅▆▆▇█
loss,█▃▂▂▁▁▁▁▁▁
x_boxes_reshape_loss,█▂▂▂▁▁▁▁▁▁
x_prob_reshape_accuracy,▁▇▇▇█▇████
x_prob_reshape_loss,█▃▂▂▁▂▁▁▁▁
epoch,9
loss,0.11011
x_boxes_reshape_loss,0.04545
x_prob_reshape_accuracy,0.97207
x_prob_reshape_loss,0.06466
